#### Import the required libraries

In [ ]:
import psycopg2 as psy
import pandas as pd
import json
import os
import time

#### Establish connection with the PostgreSQL database

In [ ]:
database_host = "localhost"
database_name = "dbl_data_challenge"
database_user = "admin"
database_pass = "vZtbqKNXGz27cQCH"

In [ ]:
con = psy.connect(
    host = database_host,
    database = database_name,
    user = database_user,
    password = database_pass
)
cur = con.cursor()

#### A dictionary containing all airlines and their twitter id

In [ ]:
airlines = {
    "KLM": "56377143",
    "AirFrance": "106062176",
    "British_Airways": "18332190",
    "AmericanAir": "22536055",
    "Lufthansa": "124476322",
    "AirBerlin": "26223583",
    "AirBerlin assist": "2182373406",
    "easyJet": "38676903",
    "RyanAir": "1542862735",
    "SingaporeAir": "253340062",
    "Qantas": "218730857",
    "EtihadAirways": "45621423",
    "VirginAtlantic": "20626359"
}

#### Create talble which will contain root group pairs

In [ ]:
cur.execute("""
    DROP TABLE IF EXISTS root_groups
""")

cur.execute("""
    CREATE TABLE root_groups (id_str VARCHAR(50), root_id VARCHAR(50), level INT)
""")

In [ ]:
cur.execute("""
    SELECT * FROM replies
""")

data = cur.fetchall()

reply_pairs = {}

for row in data: 
    if row[0] in reply_pairs: 
        reply_pairs[row[0]].append(row[1])
    else: 
        reply_pairs[row[0]] = [row[1]]

#### Get all the rout groups by iterating over all the tweets made by airlines which are not a reply

In [ ]:
for airline in airlines: 
    start = time.time()
    print("Getting root groups by "+airline)
    airline_id = airlines[airline]
    cur.execute("""
        SELECT id_str FROM tweets WHERE user_id_str = %(airline_id)s AND in_reply_to_status_id_str IS NULL
    """, {"airline_id": airline_id})
    data = cur.fetchall()
    print(str(len(data))+" possible root groups found")
    for idx, row in enumerate(data): 
        a1 = row[0]
        try: 
            b = reply_pairs[a1]
            cur.execute("""
                INSERT INTO root_groups (id_str, root_id, level) VALUES (%(id_str)s, %(aba_id)s, %(level)s)
            """, {"id_str": a1, "aba_id": a1, "level": 1})
            for value in b: 
                cur.execute("""
                    INSERT INTO root_groups (id_str, root_id, level) VALUES (%(id_str)s, %(aba_id)s, %(level)s)
                """, {"id_str": value, "aba_id": a1, "level": 2})
        except KeyError: 
            pass
        if (idx+1) % 50 == 0: 
            print("Currently at "+str(idx+1)+"/"+str(len(data))+" in "+str(time.time()-start)+"s")
            start = time.time()
con.commit()

In [ ]:
cur.close()
con.close()